## Importing necessary libraries

In [1]:
import numpy as np
import PIL
from PIL import Image, ImageOps
import io

import torch
import torchvision

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import tkinter as tk
from tkinter import ttk, colorchooser

## Construcation of best performance Model

In [2]:
class ConvModel(nn.Module):
    
    # constructor
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.conv3 = nn.Conv2d(64, 64, 3, 1)
        self.fc1 = nn.Linear(1024, 100)
        self.fc2 = nn.Linear(100, 10)

    # feed forward function
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x, 2)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        output = F.log_softmax(self.fc2(x), dim=1)
        
        return output

## Implementation of Handwritten Digit Recognizer app

In [3]:
# application of Handwritten Digit Recognizer
# DISCLAIMER! Ghostscript must be installed and added to PATH
class HandwrittenDigitRecognizer:
    
    # constructor
    def __init__(self,master):
        self.master = master
        self.color_fg = 'black'
        self.color_bg = 'white'
        self.old_x = None
        self.old_y = None
        self.penwidth = 5
        self.create_widgets()
        self.canv.bind('<B1-Motion>', self.paint)
        self.canv.bind('<ButtonRelease-1>', self.reset)

    # drawing
    def paint(self,e):
        if self.old_x and self.old_y:
            self.canv.create_line(self.old_x, self.old_y, e.x ,e.y, width=self.penwidth, fill=self.color_fg, capstyle='round', smooth=True)

        self.old_x = e.x
        self.old_y = e.y

    #reseting x i y coordinate
    def reset(self, e): 
        self.old_x = None
        self.old_y = None      

    # changing penwidth
    def change_width(self, e): 
        self.penwidth = e

    # clearing canvas
    def clear(self):
        self.canv.delete('all')
        
        self.results.destroy()
        self.probability.destroy()
        
        self.create_labels()
    
    def create_labels(self, result = None, prob = None):
        
        result_text = 'Result: ' + (str(result) if result != None else '')
        
        # creating label for result
        self.results = tk.Frame(self.master, padx = 20, pady = -5)
        self.results_text = tk.Label(self.results, text=result_text, font=('arial 12')).grid(row=0,column=0)
        self.results.pack(side='top')
        
        prob_text = 'Probability: ' + ('{:0.2f}'.format(prob) if prob != None else '')
        
        # creating label for result probability
        self.probability = tk.Frame(self.master, padx = 30, pady = 10)
        self.probability_text = tk.Label(self.probability, text=prob_text, font=('arial 12')).grid(row=0,column=0)    
        self.probability.pack(side='bottom')

    # creating all widgets
    def create_widgets(self):
    
        # creating label for penwidth
        self.controls = tk.Frame(self.master, padx = 5, pady = 5)
        tk.Label(self.controls, text='Pen Width:', font=('arial 18')).grid(row=0,column=0)
        
        # creating slider for penwidth
        self.slider = ttk.Scale(self.controls, from_= 100, to = 5 ,command=self.change_width, orient='vertical')
        self.slider.set(self.penwidth)
        self.slider.grid(row=0, column=1, ipadx=30)
        self.controls.pack(side='left')
        
        # creating 560*560 canvas for drawing
        self.canv = tk.Canvas(self.master, width=560, height=560, bg=self.color_bg)
        self.canv.pack()
        
        # creating reset button for reseting canvas
        self.exit_button = tk.Button(self.master, text='EXIT', fg='red', command=self.master.destroy)
        self.exit_button.pack(side='left')
        
        # creating reset button for reseting canvas
        self.reset_button = tk.Button(self.master, text='RESET', fg='black', command=self.clear)
        self.reset_button.pack(side='left')
        
        # creating reset button for reseting canvas
        self.reset_button = tk.Button(self.master, text=' CHECK \n RESULTS', fg='black', command=self.check_results)
        self.reset_button.pack(side='left')
        
        self.create_labels()
        
    def get_image(self):
        # getting image from canvas
        self.canv.postscript(file='utils/digit.eps', colormode='gray', height=560, width=560)
        img = Image.open('utils/digit.eps')
        img.save('utils/digit.png', 'png')
        
        # resizing image as MNIST size
        img = Image.open('utils/digit.png')
        new_image = img.resize((28, 28))
        new_image.save('utils/digit_28.png')
        
        img = Image.open('utils/digit_28.png')
        img_gray = ImageOps.grayscale(img)
        img_gray = abs(255-np.array(img_gray, dtype='float32'))/255.0
        
        return img_gray
        
    def check_results(self):
        img = self.get_image()
        recognized_number, probability = get_results(img)
        
        self.results.destroy()
        self.probability.destroy()
        
        self.create_labels(result = recognized_number, prob = probability)

In [4]:
# function for getting results for new data
def get_results(img):
    
    with torch.no_grad():
        output = model(torch.tensor(img).view(-1,1,28,28))
        result = torch.argmax(output).numpy()
        probability = np.exp(torch.max(output).numpy())
        
    return result, probability

In [5]:
# main part of program
if __name__ == '__main__':
    root = tk.Tk()
    root.title('Handwritten Digit Recognizer')
    root.geometry('800x650')
    model = torch.load('models/mnist_cnn.pt')
    model = model.to(torch.device('cpu'))
    HandwrittenDigitRecognizer(master=root)
    root.mainloop()